## Step 1: Environment Setup and Data Download

This section covers the installation of necessary libraries and the download of the UCF50 action recognition dataset using Kagglehub. Essential libraries like `tensorflow`, `opencv-python`, and `numpy` are imported to support the video processing and deep learning tasks.

In [1]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 19.9 MB/s eta 0:00:00


In [2]:
!pip install tensorflow
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import kagglehub
path = kagglehub.dataset_download("vineethakkinapalli/ucf50-action-recognition-dataset")
# The 'path' variable contains the directory where the dataset is downloaded.
# pd.DataFrame expects structured data, not a directory path directly.
# Let's inspect the contents of the directory to see what files are available.
print(f"Contents of the downloaded dataset directory '{path}':")
print(os.listdir(path))

# You would typically read a specific file (e.g., CSV, image files) from this directory
# to create a DataFrame, depending on the dataset structure.
# For example, if there's a CSV file:
# df = pd.read_csv(os.path.join(path, 'your_file.csv'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 869.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.9 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


100%|██████████| 3.04G/3.04G [00:29<00:00, 112MB/s]

Extracting files...


Contents of the downloaded dataset directory '/root/.cache/kagglehub/datasets/vineethakkinapalli/ucf50-action-recognition-dataset/versions/1':
['UCF50']


## Step 2: Data Preprocessing and Feature Extraction

Here, we define parameters for image resizing (`IMAGE_HEIGHT`, `IMAGE_WIDTH`), sequence length (`SEQUENCE_LENGTH`), and the specific action classes to be recognized (`CLASS_LIST`). Two key functions are created:

*   `frame_extraction(video_path)`: This function extracts a fixed number of frames from a given video, resizes them, and normalizes their pixel values.
*   `create_dataset()`: This function iterates through the defined classes, extracts frames for each video, and compiles them into a dataset suitable for model training, along with their corresponding labels and video paths.

The `create_dataset()` function is called to process the downloaded videos and generate the `features` (video frames) and `labels` for our action recognition model.

The categorical labels are converted into a one-hot encoded format, which is required for training the deep learning model. The dataset is then split into training and testing sets to evaluate the model's performance on unseen data.

In [3]:
all_class_name = os.listdir(path)

In [4]:
IMAGE_HEIGHT,IMAGE_WIDTH = 64,64
SEQUENCE_LENGTH = 20
DATASET_DIR = os.path.join(path,'UCF50')

CLASS_LIST = ["WalkingWithDog", "TaiChi", "Swing", "HorseRiding"]

In [5]:
def frame_extraction(video_path):
  frame_list = []
  video_reader = cv2.VideoCapture(video_path)
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

  for frame_counter in range(SEQUENCE_LENGTH):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
    success,frame = video_reader.read()
    if not success : break
    resized_frame = cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
    normalized_frame = resized_frame/255
    frame_list.append(normalized_frame)
  video_reader.release()

  return frame_list

In [6]:
def create_dataset():

  features = []
  labels = []
  video_files_paths = []

  for class_index,class_name , in enumerate(CLASS_LIST):
    print (f'Extracting data of class : {class_name}')
    files_list = os.listdir(os.path.join(DATASET_DIR,class_name))

    for file_name in files_list:
      video_file_path = os.path.join(DATASET_DIR, class_name,file_name)
      frames = frame_extraction(video_file_path)
      if len(frames) ==   SEQUENCE_LENGTH :
        features.append(frames)
        labels.append(class_index)
        video_files_paths.append(video_file_path)

  features = np.asarray(features)
  labels = np.array(labels)

  return features,labels,video_files_paths

In [7]:
features,labels,video_file_paths = create_dataset()

Extracting data of class : WalkingWithDog
Extracting data of class : TaiChi
Extracting data of class : Swing
Extracting data of class : HorseRiding


In [8]:
#encoding of labels
one_hot_encoded_labels = to_categorical(labels)

In [9]:
#split the dataset into train and test
features_train ,features_test,label_train,label_test = train_test_split(features,one_hot_encoded_labels,test_size=0.2,shuffle=True,random_state=42)


## Step 3: Model Architecture and Training

A ConvLSTM2D model is constructed using `tensorflow.keras.models.Sequential`. This architecture is particularly suited for video data as it can learn spatial and temporal features. The model includes ConvLSTM layers, a MaxPooling3D layer, a Dropout layer for regularization, and a Dense output layer with a softmax activation for classification. The model is compiled with 'categorical_crossentropy' loss and 'Adam' optimizer.

The model is then trained using the prepared training data. An `EarlyStopping` callback is implemented to prevent overfitting by monitoring the validation loss and restoring the best weights.

In [10]:
def create_convlstm_model():
    model = Sequential()
    model.add(ConvLSTM2D(filters=4, kernel_size=(3, 3), activation='tanh', data_format="channels_last",
                         recurrent_dropout=0.2, return_sequences=True,
                         input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))

    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))

    model.add(Flatten())
    model.add(Dense(len(CLASS_LIST), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
    return model
model = create_convlstm_model()
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 20, 62, 62, 4)  │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 20, 31, 31, 4)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 20, 31, 31, 4)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 76880)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       307,524 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 308,548 (1.18 MB)

 Trainable params: 308,548 (1.18 MB)

 Non-trainable params: 0 (0.00 B)

## Step 4: Model Evaluation and Saving

After training, the model's performance is evaluated on the test set. The trained model is saved to an HDF5 file (`model.h5`) for future use, preventing the need to retrain it every time.

In [11]:
#train the model
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15,mode='min',restore_best_weights =True)
model.fit(
    features_train,
    label_train,
    epochs=15,
    batch_size=4,
    shuffle=True,
    validation_split=0.2,
    callbacks=[early_stopping_callback]
)

Epoch 1/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 29s 301ms/step - accuracy: 0.3003 - loss: 4.1798 - val_accuracy: 0.4270 - val_loss: 1.5894
Epoch 2/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 27s 309ms/step - accuracy: 0.5501 - loss: 1.0075 - val_accuracy: 0.5955 - val_loss: 0.9284
Epoch 3/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 27s 299ms/step - accuracy: 0.8131 - loss: 0.5882 - val_accuracy: 0.6517 - val_loss: 0.8455
Epoch 4/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 27s 299ms/step - accuracy: 0.9111 - loss: 0.3626 - val_accuracy: 0.7079 - val_loss: 0.7457
Epoch 5/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 26s 296ms/step - accuracy: 0.9838 - loss: 0.1850 - val_accuracy: 0.7303 - val_loss: 0.8324
Epoch 6/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 26s 296ms/step - accuracy: 0.9896 - loss: 0.1067 - val_accuracy: 0.6517 - val_loss: 1.4529
Epoch 7/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 26s 295ms/step - accuracy: 0.9720 - loss: 0.1214 - val_accuracy: 0.6629 - val_loss: 1.1093
Epoch 8/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 29s 325ms/step - accuracy: 1.0000 - loss: 0.0514 - val_accu

The model has been saved as `model.h5`.

In [13]:
loss,acc = model.evaluate(features_test,label_test)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 735ms/step - accuracy: 0.6646 - loss: 0.9740


In [12]:
model.save('model.h5')

The `predict_on_video` function is designed to take a video file path as input, extract and preprocess its frames, and then use the trained model to predict the action in real-time. The predicted action class name is overlaid on the video frames, and the modified video is saved to an output file. This demonstrates the model's capability to perform inference on new, unseen video data.

A random video from the dataset is selected, and the `predict_on_video` function is called to process it, demonstrating the end-to-end action recognition pipeline. The resulting video with overlaid predictions is saved, allowing for visual inspection of the model's performance.

The video `output_action_recognition.mp4` has been generated and saved in your Colab environment. You can download this file from the file browser on the left sidebar to view the action recognition results.

In [17]:
def predict_on_video(video_file_path,output_file_path,SEQUENCE_LENGTH):
  video_reader = cv2.VideoCapture(video_file_path)
  orignal_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
  orignal_video_heigh = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))


  # Corrected cv2.VideoWriter call: fixed typo and added missing parenthesis
  fourcc = cv2.VideoWriter_fourcc(*'MP4V') # Use *'MP4V' for cross-platform compatibility
  video_writer = cv2.VideoWriter(output_file_path, fourcc,
                                 video_reader.get(cv2.CAP_PROP_FPS), (orignal_video_width,orignal_video_heigh))
  # Corrected deque initialization: removed extra parenthesis
  frames_queue = deque(maxlen = SEQUENCE_LENGTH)
  predicted_class_name = ''

  while video_reader.isOpened():
    ok,frame = video_reader.read()
    if not ok:
      break

    # Corrected indentation for these lines
    resized_frame = cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))

    normalized_frame = resized_frame/255

    frames_queue.append(normalized_frame)

    if len(frames_queue) == SEQUENCE_LENGTH:
      # Corrected model.predict call: added missing parenthesis and removed unnecessary [0]
      predicted_labels_probabilities = model.predict(np.expand_dims(frames_queue,axis = 0))

      predicted_labels = np.argmax(predicted_labels_probabilities)
      predicted_class_name = CLASS_LIST[predicted_labels]

      cv2.putText(frame,predicted_class_name,(10,30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)

    # Moved video_writer.write to ensure all frames are written
    video_writer.write(frame)
  video_reader.release()
  video_writer.release() # Added release for video_writer

In [19]:
import random

# Select a random video from the extracted video files
random_video_path = random.choice(video_file_paths)

# Define the output video path
output_video_path = "output_action_recognition.mp4"

# Perform action recognition on the video
predict_on_video(random_video_path, output_video_path, SEQUENCE_LENGTH)

print(f"Action recognition complete. Output video saved to: {output_video_path}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━